# Обучение на гетерогенных графах

In [1]:
import torch

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def set_seed():
    """
    Задает стартовое значение генератора псевдослучайных
    чисел для воспроизводимости.
    """
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)

In [2]:
from torch.nn import Linear
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, dim_in, dim_h):
        super().__init__(aggr='add')
        self.linear = Linear(dim_in, dim_h, bias=False)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, 
                                       num_nodes=x.size(0))

        x = self.linear(x)

        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        out = self.propagate(edge_index, x=x, norm=norm)

        return out

    def message(self, x, norm):
        return norm.view(-1, 1) * x

2024-02-15 15:03:05.233126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
conv = GCNConv(16, 32)

## Гетерогенные графы

In [4]:
from torch_geometric.data import HeteroData
data = HeteroData()

# [num_users, num_features_users]
data['user'].x = torch.Tensor([[1, 1, 1, 1], 
                               [2, 2, 2, 2], 
                               [3, 3, 3, 3]]) 
data['game'].x = torch.Tensor([[1, 1], [2, 2]])
data['dev'].x = torch.Tensor([[1], [2]])

# [2, num_edges_follows]
data['user', 'follows', 'user'].edge_index = torch.Tensor([[0, 1], 
                                                           [1, 2]]) 
data['user', 'plays', 'game'].edge_index = torch.Tensor([[0, 1, 1, 2], 
                                                         [0, 0, 1, 1]])
data['dev', 'develops', 'game'].edge_index = torch.Tensor([[0, 1], 
                                                           [0, 1]])

data['user', 'plays', 'game'].edge_attr = torch.Tensor(
    [[2], [0.5], [10], [12]]
)

data

HeteroData(
  user={ x=[3, 4] },
  game={ x=[2, 2] },
  dev={ x=[2, 1] },
  (user, follows, user)={ edge_index=[2, 2] },
  (user, plays, game)={
    edge_index=[2, 4],
    edge_attr=[4, 1]
  },
  (dev, develops, game)={ edge_index=[2, 2] }
)

In [5]:
from torch import nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import DBLP
from torch_geometric.nn import GAT

In [6]:
metapaths = [[('author', 'paper'), ('paper', 'author')]]
transform = T.AddMetaPaths(metapaths=metapaths, 
                           drop_orig_edge_types=True)
dataset = DBLP('.', transform=transform)
data = dataset[0]
print(data)

HeteroData(
  metapath_dict={ (author, metapath_0, author)=[2] },
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={ num_nodes=20 },
  (author, metapath_0, author)={ edge_index=[2, 11113] }
)


/Users/artemgruzdev/anaconda3/lib/python3.10/site-packages/torch_sparse/matmul.py:97: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  C = torch.sparse.mm(A, B)


In [7]:
set_seed()

model = GAT(in_channels=-1, 
            hidden_channels=64, 
            out_channels=4, 
            num_layers=1)

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=0.001, 
                             weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict['author'],
                 data.edge_index_dict[
                     ('author', 'metapath_0', 'author')
                 ]).argmax(dim=-1)
    acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
    return float(acc)

for epoch in range(101):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict['author'], 
                data.edge_index_dict[
                    ('author', 'metapath_0', 'author')
                ])
    mask = data['author'].train_mask
    loss = F.cross_entropy(out[mask], 
                           data['author'].y[mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        train_acc = test(data['author'].train_mask)
        val_acc = test(data['author'].val_mask)
        print(f'Эпоха {epoch:>3}:\n| Функция потерь на обуч. выборке: '
              f'{loss:.4f} | Правильность на обуч. выборке: '
              f'{train_acc*100:.2f}% \n| Правильность на валид. '
              f'выборке: {val_acc*100:.2f}%')


test_acc = test(data['author'].test_mask)
print(f'Качество на тестовом наборе: {test_acc*100:.2f}%')

Эпоха   0:
| Функция потерь на обуч. выборке: 1.3973 | Правильность на обуч. выборке: 28.00% 
| Правильность на валид. выборке: 21.50%
Эпоха  20:
| Функция потерь на обуч. выборке: 1.2426 | Правильность на обуч. выборке: 49.50% 
| Правильность на валид. выборке: 40.75%
Эпоха  40:
| Функция потерь на обуч. выборке: 1.1210 | Правильность на обуч. выборке: 66.00% 
| Правильность на валид. выборке: 56.50%
Эпоха  60:
| Функция потерь на обуч. выборке: 1.0224 | Правильность на обуч. выборке: 77.25% 
| Правильность на валид. выборке: 61.50%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.9393 | Правильность на обуч. выборке: 81.50% 
| Правильность на валид. выборке: 67.00%
Эпоха 100:
| Функция потерь на обуч. выборке: 0.8676 | Правильность на обуч. выборке: 83.25% 
| Правильность на валид. выборке: 68.75%
Качество на тестовом наборе: 71.60%


In [8]:
from torch_geometric.nn import GATConv, Linear, to_hetero

dataset = DBLP(root='.')
data = dataset[0]

data['conference'].x = torch.zeros(20, 1)

class GAT(torch.nn.Module):
    def __init__(self, dim_h, dim_out):
        super().__init__()
        self.conv = GATConv((-1, -1), dim_h, add_self_loops=False)
        self.linear = nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.conv(x, edge_index).relu()
        h = self.linear(h)
        return h

model = GAT(dim_h=64, dim_out=4)
model = to_hetero(model, data.metadata(), aggr='sum')
print(model)

GraphModule(
  (conv): ModuleDict(
    (author__to__paper): GATConv((-1, -1), 64, heads=1)
    (paper__to__author): GATConv((-1, -1), 64, heads=1)
    (paper__to__term): GATConv((-1, -1), 64, heads=1)
    (paper__to__conference): GATConv((-1, -1), 64, heads=1)
    (term__to__paper): GATConv((-1, -1), 64, heads=1)
    (conference__to__paper): GATConv((-1, -1), 64, heads=1)
  )
  (linear): ModuleDict(
    (author): Linear(in_features=64, out_features=4, bias=True)
    (paper): Linear(in_features=64, out_features=4, bias=True)
    (term): Linear(in_features=64, out_features=4, bias=True)
    (conference): Linear(in_features=64, out_features=4, bias=True)
  )
)



def forward(self, x, edge_index):
    x_dict = torch_geometric_nn_to_hetero_transformer_get_dict(x);  x = None
    x__author = x_dict.get('author', None)
    x__paper = x_dict.get('paper', None)
    x__term = x_dict.get('term', None)
    x__conference = x_dict.get('conference', None);  x_dict = None
    edge_index_dict = torch_ge

In [9]:
set_seed()

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=0.001, 
                             weight_decay=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict, 
                 data.edge_index_dict)['author'].argmax(dim=-1)
    acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
    return float(acc)

for epoch in range(101):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)['author']
    mask = data['author'].train_mask
    loss = F.cross_entropy(out[mask], data['author'].y[mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        train_acc = test(data['author'].train_mask)
        val_acc = test(data['author'].val_mask)
        print(f'Эпоха {epoch:>3}:\n| Функция потерь на обуч. выборке: '
              f'{loss:.4f} | Правильность на обуч. выборке: '
              f'{train_acc*100:.2f}% \n| Правильность на валид. '
              f'выборке: {val_acc*100:.2f}%')

test_acc = test(data['author'].test_mask)
print(f'Качество на тестовом наборе: {test_acc*100:.2f}%')

Эпоха   0:
| Функция потерь на обуч. выборке: 1.3999 | Правильность на обуч. выборке: 17.25% 
| Правильность на валид. выборке: 23.00%
Эпоха  20:
| Функция потерь на обуч. выборке: 1.2210 | Правильность на обуч. выборке: 97.25% 
| Правильность на валид. выборке: 73.00%
Эпоха  40:
| Функция потерь на обуч. выборке: 0.8867 | Правильность на обуч. выборке: 96.25% 
| Правильность на валид. выборке: 69.75%
Эпоха  60:
| Функция потерь на обуч. выборке: 0.5288 | Правильность на обуч. выборке: 98.25% 
| Правильность на валид. выборке: 72.75%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.2768 | Правильность на обуч. выборке: 99.25% 
| Правильность на валид. выборке: 75.00%
Эпоха 100:
| Функция потерь на обуч. выборке: 0.1506 | Правильность на обуч. выборке: 100.00% 
| Правильность на валид. выборке: 75.25%
Качество на тестовом наборе: 78.11%


## Иерархическая нейронная сеть с самовниманием (HAN)

In [10]:
set_seed()

from torch_geometric.nn import HANConv

dataset = DBLP('.')
data = dataset[0]

data['conference'].x = torch.zeros(20, 1)

class HAN(nn.Module):
    def __init__(self, dim_in, dim_out, dim_h=128, heads=8):
        super().__init__()
        self.han = HANConv(dim_in, dim_h, heads=heads, 
                           dropout=0.6, metadata=data.metadata())
        self.linear = nn.Linear(dim_h, dim_out)

    def forward(self, x_dict, edge_index_dict):
        out = self.han(x_dict, edge_index_dict)
        out = self.linear(out['author'])
        return out

model = HAN(dim_in=-1, dim_out=4)

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=0.001, 
                             weight_decay=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)
    acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
    return float(acc)

for epoch in range(101):
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['author'].train_mask
    loss = F.cross_entropy(out[mask], data['author'].y[mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        train_acc = test(data['author'].train_mask)
        val_acc = test(data['author'].val_mask)
        print(f'Эпоха {epoch:>3}:\n| Функция потерь на обуч. выборке: '
              f'{loss:.4f} | Правильность на обуч. выборке: '
              f'{train_acc*100:.2f}% \n| Правильность на валид. '
              f'выборке: {val_acc*100:.2f}%')

test_acc = test(data['author'].test_mask)
print(f'Качество на тестовом наборе: {test_acc*100:.2f}%')

Эпоха   0:
| Функция потерь на обуч. выборке: 1.3931 | Правильность на обуч. выборке: 17.50% 
| Правильность на валид. выборке: 23.00%
Эпоха  20:
| Функция потерь на обуч. выборке: 1.1545 | Правильность на обуч. выборке: 93.50% 
| Правильность на валид. выборке: 69.75%
Эпоха  40:
| Функция потерь на обуч. выборке: 0.7783 | Правильность на обуч. выборке: 95.75% 
| Правильность на валид. выборке: 70.50%
Эпоха  60:
| Функция потерь на обуч. выборке: 0.4627 | Правильность на обуч. выборке: 98.25% 
| Правильность на валид. выборке: 75.75%
Эпоха  80:
| Функция потерь на обуч. выборке: 0.2935 | Правильность на обуч. выборке: 99.50% 
| Правильность на валид. выборке: 77.50%
Эпоха 100:
| Функция потерь на обуч. выборке: 0.2152 | Правильность на обуч. выборке: 100.00% 
| Правильность на валид. выборке: 78.50%
Качество на тестовом наборе: 81.98%
